# 利用arcpy包求解BC半径形状指数

## Boyce-Clark半径形状指数：

### 1964年，Boyce和Clark提出的放射状指数。
### 其基本思想是将研究区形状与标准圆进行比较，得出一个相对指数。
### 这种方法是一种基于半径测度的，又叫半径形状指数。
### 此指数用于研究几何图形是否规则，越接近圆形，指数越小（圆的B-C指数为0）


## 代码及说明 

### 导入arcpy包

In [1]:
import arcpy

### 设置数据处理环境，包括工作空间和允许数据覆盖，定义空间参考

In [2]:
arcpy.env.workspace = r"E:\Other\1\data\PythonDemo\PythonDemo.gdb"
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(4326)

### 创建结果图层，并且添加需要的字段

In [3]:
pline = arcpy.CreateFeatureclass_management(arcpy.env.workspace, "sline", "POLYLINE", spatial_reference = sr)
arcpy.AddField_management(pline, "name", "TEXT")
bcipnt = arcpy.CreateFeatureclass_management(arcpy.env.workspace, "bci", "POINT", spatial_reference = sr)
arcpy.AddField_management(bcipnt, "name", "TEXT")
arcpy.AddField_management(bcipnt, "bci", "FLOAT")

<Result 'E:\\Other\\1\\data\\PythonDemo\\PythonDemo.gdb\\bci'>

### 定义各种方法

#### 子方法1 ： 获取从中心点到extent的最大范围的方法

In [4]:
def maxdist(pnt, ext):
    point = arcpy.Point(ext.XMin, ext.YMin)
    pg1 = arcpy.PointGeometry(point, sr)
    dist1 = pnt.angleAndDistanceTo(pg1)[1]
    point = arcpy.Point(ext.XMax, ext.YMax)
    pg2 = arcpy.PointGeometry(point, sr)
    dist2 = pnt.angleAndDistanceTo(pg2)[1]
    return max(dist1, dist2)

#### 子方法2：根据角度和距离，创建一条射线

In [5]:
def createInPnt(cent, dist, angle):
    adpnt = cent.pointFromAngleAndDistance(angle, dist)
    line = arcpy.Polyline(arcpy.Array([cent.centroid, adpnt.centroid]), sr)
    return line

#### 子方法3：如果有多个交点，有两种处理方法，一种是取短线，一种是取长线

In [6]:
def minPoint(cent, pnt):
    p = pnt.centroid
    dmin = cent.angleAndDistanceTo(arcpy.PointGeometry(p, sr))[1]
    for i in range(pnt.pointCount):
        dist1 = cent.angleAndDistanceTo(arcpy.PointGeometry(pnt[i], sr))[1]
        if dmin > dist1:
            dmin = dist1
            p = pnt[i]
    return (dmin, p)

def maxPoint(cent, pnt):
    dmax = 0
    p = pnt.centroid
    for i in range(pnt.pointCount):
        dist1 = cent.angleAndDistanceTo(arcpy.PointGeometry(pnt[i], sr))[1]
        if dmax < dist1:
            dmax = dist1
            p = pnt[i]
    return (dmax, p)

#### 子方法4：将生成的结果的线，写入到数据中

In [7]:
def createPline(name, line):
    cursor = arcpy.da.InsertCursor("sline", ["SHAPE@", "name"])
    for l in line:
        row = (l, name)
        cursor.insertRow(row)
    del cursor

#### 子方法5：计算BC指数

In [8]:
def culabcIndex(distlist, n):
    distsum = sum(distlist)
    bci = 0.0
    for d in distlist:
        bci += math.fabs(((d / distsum) * 100) - (100.0 / n))
    return bci

#### 子方法6: 主功能

In [9]:
def bcindex(pline, name, poly, n):
    cent = arcpy.PointGeometry(poly.centroid, sr)
    angle = 360.0 / n
    ext = poly.extent
    dist = maxdist(cent, ext)
    distlist = []
    linelist = []
    for i in range(n):
        a = i * angle
        line = createInPnt(cent, dist, a)
        pnt = line.intersect(poly, 1)
        mp = maxPoint(cent, pnt)
        linelist.append(arcpy.Polyline(arcpy.Array([cent.centroid, mp[1]]), sr))
        distlist.append(mp[0])
    bci = culabcIndex(distlist, n)
    createPline(name, linelist)
    return bci

#### main函数

In [10]:
fields = ['SHAPE@', 'name']
biclist = []
geo = [row for row in arcpy.da.SearchCursor("cn", fields)]
for g in geo:
    bci = bcindex(pline, g[1], g[0], 36)
    biclist.append((g[0].centroid, g[1], bci))
bcicursor = arcpy.da.InsertCursor(bcipnt, ["SHAPE@", "name", "bci"])
for b in biclist:
    print(b)
    row = b
    bcicursor.insertRow(row)
del bcicursor

(<Point (127.72624359266773, 47.84220356013497, #, #)>, '黑龙江', 29.884698351976635)
(<Point (85.20062883483251, 41.11242032354995, #, #)>, '新疆', 19.13302859943545)
(<Point (112.26308765476394, 37.57003564328917, #, #)>, '山西', 28.395275857599124)
(<Point (106.15840294004676, 37.26834054240205, #, #)>, '宁夏', 33.60425733789232)
(<Point (88.45982362589172, 31.495833747271853, #, #)>, '西藏', 31.864406467251175)
(<Point (118.10740222691372, 36.32194184409586, #, #)>, '山东', 20.48993063729425)
(<Point (113.5811856659696, 33.874427886351505, #, #)>, '河南', 17.69485721600139)
(<Point (119.42019804373498, 32.98083427554814, #, #)>, '江苏', 26.1252011768546)
(<Point (117.19721898525943, 31.822978603795995, #, #)>, '安徽', 21.148567965123675)
(<Point (112.23941749142321, 30.967806163742644, #, #)>, '湖北', 31.119986172013608)
(<Point (120.02375899354038, 29.165944834160364, #, #)>, '浙江', 10.77822823519536)
(<Point (115.69996028206573, 27.608126264151316, #, #)>, '江西', 23.675399835449145)
(<Point (111.694493